In [1]:
!pip install imblearn

In [14]:
import torch
import torch.nn as nn
from imblearn.over_sampling import SMOTE
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from torchvision import datasets, transforms, utils
from torch.utils import data

In [15]:
df=pd.read_excel('./modeling_data/Result2.xlsx')

X=df.iloc[:,1:7]
y_origin=df['Label']

train_x=X[:16047]
test_x=X[16047:]

train_y=y_origin[:16047]
test_y=y_origin[16047:]


smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(train_x,train_y)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', train_x.shape, train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(train_y).value_counts())
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

ms=MinMaxScaler()
X_train_over_ms=ms.fit_transform(X_train_over)
X_test_ms=ms.fit_transform(test_x)

y_train_over_dummies=pd.get_dummies(y_train_over)
test_y_dummies=pd.get_dummies(test_y)

train_x = X_train_over.values
test_x = X_test_ms
train_y = y_train_over_dummies.values
test_y=test_y_dummies.values

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (16047, 6) (16047,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (24510, 6) (24510,)
SMOTE 적용 전 레이블 값 분포: 
 -1    8170
 1    7731
 0     146
Name: Label, dtype: int64
SMOTE 적용 후 레이블 값 분포: 
  0    8170
 1    8170
-1    8170
Name: Label, dtype: int64


In [20]:
train_x_torch=torch.FloatTensor(train_x)
test_x_torch=torch.FloatTensor(test_x)

train_y_torch=torch.FloatTensor(train_y)
test_y_torch=torch.FloatTensor(test_y)

In [21]:
train_y_torch

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])

In [22]:
# 데이터 로더 준비
batch_size = 16

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 30
BATCH_SIZE = 71


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)

    def forward(self, x):
        x = x.view(-1, 6)  # 1차원 행렬로 변환
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
epoches=300
train_y_torch=train_y_torch.squeeze(dim=-1)

In [23]:
for epoche in range(epoches+1):
    pred=model(train_x_torch)

    loss=F.cross_entropy(pred, train_y_torch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# def train(model, train_loader, optimizer):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(DEVICE), target.to(DEVICE)
#         optimizer.zero_grad()  # 기울기 초기화
#         output = model(data)
#         loss = F.cross_entropy(output, target)
#         loss.backward()
#         optimizer.step()


# def evaluate(model, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(DEVICE), target.to(DEVICE)
#             output = model(data)

#             test_loss += F.cross_entropy(output, target, reduction='sum').item()  # 오차 누적값

#             pred = output.max(1, keepdim=True)[1]  # output의 가장 큰 값의 인덱스 (argmax)
#             correct += pred.eq(target.view_as(pred)).sum().item()  # target, pred 일치하는 값의 개수

#         test_loss /= len(test_loader.dataset)  # 오차의 평균값
#         test_accuracy = 100 * correct / len(test_loader.dataset)
#         return test_loss, test_accuracy


# for epoch in range(1, EPOCHS + 1):
#     train(model, train_loader, optimizer)
#     test_loss, test_accuracy = evaluate(model, test_loader)

#     print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy:{test_accuracy:.2f}')

## iris 붓꽃데이터를 이용한 DNN

In [41]:
import torch
import torch.nn as nn
from imblearn.over_sampling import SMOTE
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from torchvision import datasets, transforms, utils
from torch.utils import data

In [42]:
import pandas as pd

from sklearn.datasets import load_iris

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [43]:
df=pd.read_excel('./modeling_data/Result2.xlsx')

X=df.iloc[:,1:7]
y_origin=df['Label']

train_x=X[:16047]
test_x=X[16047:]

train_y=y_origin[:16047]
test_y=y_origin[16047:]


smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(train_x,train_y)

print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', train_x.shape, train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)   
print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(train_y).value_counts())
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

ms=MinMaxScaler()
X_train_over_ms=ms.fit_transform(X_train_over)
X_test_ms=ms.fit_transform(test_x)

y_train_over_dummies=pd.get_dummies(y_train_over)
test_y_dummies=pd.get_dummies(test_y)

train_x = X_train_over.values
test_x = X_test_ms
train_y = y_train_over_dummies.values
test_y=test_y_dummies.values

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (16047, 6) (16047,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (24510, 6) (24510,)
SMOTE 적용 전 레이블 값 분포: 
 -1    8170
 1    7731
 0     146
Name: Label, dtype: int64
SMOTE 적용 후 레이블 값 분포: 
  0    8170
 1    8170
-1    8170
Name: Label, dtype: int64


In [44]:
X=X.to_numpy()

In [45]:
y_origin=y_origin.to_numpy()

In [46]:
dataset = load_iris()

data = dataset.data
label = dataset.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_origin, test_size=0.25)

In [47]:
# DataLoader 생성
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [48]:
train_set = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=71, shuffle=True)

In [49]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer0 = nn.Linear(4, 128)
        self.layer1 = nn.Linear(128, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.layer4 = nn.Linear(16, 3)

        self.bn0 = nn.BatchNorm1d(128) # 정규화
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)

        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.bn0(self.layer0(x)))
        x = self.act(self.bn1(self.layer1(x)))
        x = self.act(self.bn2(self.layer2(x)))
        x = self.act(self.layer3(x))
        x = self.layer4(x)
        return x

In [50]:
net = Net()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 200

In [56]:
losses = list()
accuracies = list()

for epoch in range(epochs):      
    epoch_loss = 0  
    epoch_accuracy = 0
    for X, y in train_loader:
  
        optimizer.zero_grad()

        output = net(X)

        loss = criterion(output, y)
        loss.backward()

        optimizer.step()
        # output = [0.11, 0.5, 0.8]  --> 예측 클래스 값
        _, predicted = torch.max(output, dim=1) # 최대값의 index를 뱉는다.
        accuracy = (predicted == y).sum().item()
        epoch_loss += loss.item()
        epoch_accuracy += accuracy
  

epoch_loss /= len(train_loader)
epoch_accuracy /= len(X_train)
print("epoch :{}, \tloss :{}, \taccuracy :{}".format(str(epoch+1).zfill(3),round(epoch_loss,4), round(epoch_accuracy,4)))
  
losses.append(epoch_loss)
accuracies.append(epoch_accuracy)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (71x6 and 4x128)

Test

In [12]:
output = net(X_test)
print(torch.max(output, dim=1))
_, predicted = torch.max(output, dim=1)
accuracy = round((predicted == y_test).sum().item() / len(y_test),4)                         
print("test_set accuracy :", round(accuracy,4))

torch.return_types.max(
values=tensor([2.9190, 0.4288, 1.2929, 0.9612, 1.4929, 1.4168, 1.1381, 1.1718, 1.3884,
        1.4329, 0.9542, 1.3164, 1.7034, 0.6331, 1.4328, 0.6363, 0.9851, 1.2574,
        1.4635, 3.4482, 0.9393, 3.3632, 1.0393, 0.9752, 0.6531, 3.3444, 1.0843,
        4.0011, 0.7095, 1.4996, 1.4197, 0.8723, 2.4232, 1.5605, 1.3377, 2.1817,
        0.9349, 1.4164], grad_fn=<MaxBackward0>),
indices=tensor([0, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 2, 1, 0, 2, 0, 1, 1,
        2, 0, 1, 0, 1, 2, 1, 2, 0, 1, 1, 0, 1, 1]))
test_set accuracy : 0.8947
